In [1]:
import pandas as pd
import numpy as np

import sklearn.feature_extraction.text as text
from sklearn import decomposition

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import pickle

import nltk
from nltk.tokenize import wordpunct_tokenize
import pickle

/Users/matthewvitha/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewvitha/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewvitha/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewvitha/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewvitha/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [7]:
import numpy as np
import pandas as pd

from collections import defaultdict, Counter

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
plt.rcParams['figure.figsize'] = (12, 10)

import sklearn
from sklearn import svm, datasets
from sklearn.utils import shuffle
from sklearn import cross_validation, metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.metrics import auc, roc_auc_score, roc_curve, precision_recall_curve, fbeta_score, make_scorer, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
import imblearn.over_sampling
from scipy import interp

random_state = np.random.RandomState(42)

/Users/matthewvitha/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/matthewvitha/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewvitha/anaconda/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
pcepi_lag = pickle.load( open('pcepi_lag','rb'))

In [37]:
print(pcepi_lag_sort.shape)
pcepi_lag_sort = pcepi_lag.sort_values(by=['date2'])


(440, 29)


In [38]:
pcepi_lag_sort.head(3)

,date2,PCEPI_PCH,FEDFUNDS_PCH,INDPRO_PCH,T10Y2YM_PCH,topic0,topic1,topic2,topic3,topic4,...,topic3_lag,topic4_lag,topic5_lag,topic6_lag,topic7_lag,topic8_lag,topic9_lag,FEDFUNDS_PCH_lag,INDPRO_PCH_lag,T10Y2YM_PCH_lag
0,1982-01,0.6,6.9,-2.0,-95.3,0.011099,0.012358,0.015551,0.005772,0.014494,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1982-02,0.3,11.8,2.0,-2050.0,0.006406,0.013169,0.018591,0.008598,0.008852,...,0.005772,0.014494,0.005591,0.013190,0.015621,0.011621,0.009398,6.9,-2.0,-95.3
2,1982-03,0.3,-0.7,-0.8,-15.4,0.009945,0.012897,0.011588,0.004926,0.007485,...,0.008598,0.008852,0.010766,0.023805,0.013006,0.013584,0.011436,11.8,2.0,-2050.0


In [23]:
['PCEPI_PCH_lag','topic0_lag','topic1_lag','topic2_lag','topic3_lag','topic4_lag','topic5_lag','topic6_lag','topic7_lag','topic8_lag','topic9_lag','FEDFUNDS_PCH_lag','INDPRO_PCH_lag','T10Y2YM_PCH_lag']

['PCEPI_PCH_lag',
 'topic0_lag',
 'topic1_lag',
 'topic2_lag',
 'topic3_lag',
 'topic4_lag',
 'topic5_lag',
 'topic6_lag',
 'topic7_lag',
 'topic8_lag',
 'topic9_lag',
 'FEDFUNDS_PCH_lag',
 'INDPRO_PCH_lag',
 'T10Y2YM_PCH_lag']

In [67]:
['date2','PCEPI_PCH_lag',
 'topic0_lag',
 'topic1_lag',
 'topic2_lag',
 'topic3_lag',
 'topic4_lag',
 'topic5_lag',
 'topic6_lag',
 'topic7_lag',
 'topic8_lag',
 'topic9_lag',
 'FEDFUNDS_PCH_lag',
 'INDPRO_PCH_lag',
 'T10Y2YM_PCH_lag']

['date2',
 'PCEPI_PCH_lag',
 'topic0_lag',
 'topic1_lag',
 'topic2_lag',
 'topic3_lag',
 'topic4_lag',
 'topic5_lag',
 'topic6_lag',
 'topic7_lag',
 'topic8_lag',
 'topic9_lag',
 'FEDFUNDS_PCH_lag',
 'INDPRO_PCH_lag',
 'T10Y2YM_PCH_lag']

In [39]:
pcepi_lag_sort_train = pcepi_lag_sort[0:220]
pcepi_lag_sort_valid = pcepi_lag_sort[221:330]
pcepi_lag_sort_holdout = pcepi_lag_sort[331:440]

In [56]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X_train = pcepi_lag_sort_train.dropna()[['PCEPI_PCH_lag','FEDFUNDS_PCH_lag','INDPRO_PCH_lag','T10Y2YM_PCH_lag']]
y_train = pcepi_lag_sort_train.dropna()[["PCEPI_PCH"]]
model.fit(X_train, y_train)
model.coef_, model.intercept_


(array([[ 3.65983734e-01,  5.38366628e-03, -6.95947507e-03,
          5.07596520e-05]]), array([0.15453442]))

In [57]:
model.score(X_train,y_train)

0.15606056477906327

In [63]:
print(X_train.shape)
print(y_train.shape)

(217, 4)
(217, 1)


In [58]:
X_valid = pcepi_lag_sort_valid.dropna()[['PCEPI_PCH_lag','FEDFUNDS_PCH_lag','INDPRO_PCH_lag','T10Y2YM_PCH_lag']]
y_valid = pcepi_lag_sort_valid.dropna()[["PCEPI_PCH"]]

In [59]:
y_pred1 = model.predict(X_valid)

In [65]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_valid, y_valid)
model.coef_, model.intercept_


(array([[3.50257149e-01, 2.72972268e-03, 3.00064776e-02, 3.73870731e-05]]),
 array([0.1251951]))

In [66]:
model.score(X_valid,y_valid)

0.2113494450120108

**Above represents a baseline model for inflation w/o NLP variables**

In [68]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X_train = pcepi_lag_sort_train.dropna()[['PCEPI_PCH_lag','FEDFUNDS_PCH_lag','INDPRO_PCH_lag','T10Y2YM_PCH_lag','topic0_lag',
 'topic1_lag',
 'topic2_lag',
 'topic3_lag',
 'topic4_lag',
 'topic5_lag',
 'topic6_lag',
 'topic7_lag',
 'topic8_lag',
 'topic9_lag',]]
y_train = pcepi_lag_sort_train.dropna()[["PCEPI_PCH"]]
model.fit(X_train, y_train)
model.coef_, model.intercept_

(array([[ 2.77150034e-01,  7.00691261e-03,  7.89377800e-04,
          8.34299275e-05,  1.34077765e+00,  2.67913633e-01,
          1.47371876e+00, -5.93712481e+00, -4.36420711e+00,
          1.59065378e+00,  8.18076754e-01, -8.84019794e-01,
         -3.50413705e-01,  1.25466116e+00]]), array([0.2260904]))

In [69]:
model.score(X_train,y_train)

0.2428123290845634

In [70]:
X_valid1 = pcepi_lag_sort_valid.dropna()[['PCEPI_PCH_lag','FEDFUNDS_PCH_lag','INDPRO_PCH_lag','T10Y2YM_PCH_lag','topic0_lag',
 'topic1_lag',
 'topic2_lag',
 'topic3_lag',
 'topic4_lag',
 'topic5_lag',
 'topic6_lag',
 'topic7_lag',
 'topic8_lag',
 'topic9_lag']]
y_valid1 = pcepi_lag_sort_valid.dropna()[["PCEPI_PCH"]]

In [71]:
import numpy as np
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_valid1, y_valid1)
model.coef_, model.intercept_

(array([[ 3.40732323e-01,  1.65284807e-03,  1.22763538e-03,
          1.03739618e-04, -2.83336139e+00, -3.35667674e+00,
         -3.29883951e+00,  3.96454883e-01, -1.17262876e+00,
          3.66875544e+00, -4.22061504e+00, -6.70310602e-01,
         -7.49054946e+00,  4.80675371e-01]]), array([0.3870246]))

In [73]:
model.score(X_valid1,y_valid1)

0.25183437474946135